In [1]:
import cv2
import os

# Path to the video file
video_path = 'vdo.mp4'

# Create an output directory to save frames if it doesn't already exist
output_dir = 'extracted_frames'
os.makedirs(output_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)

# Frame extraction
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    
    # Break the loop if there are no frames left to read
    if not ret:
        break

    # Save the frame as an image file
    frame_filename = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(frame_filename, frame)
    
    # Print progress
    print(f"Extracted {frame_filename}")
    
    # Increment frame count
    frame_count += 1

# Release the video capture object
cap.release()
print("Frame extraction complete.")


Extracted extracted_frames\frame_0000.jpg
Extracted extracted_frames\frame_0001.jpg
Extracted extracted_frames\frame_0002.jpg
Extracted extracted_frames\frame_0003.jpg
Extracted extracted_frames\frame_0004.jpg
Extracted extracted_frames\frame_0005.jpg
Extracted extracted_frames\frame_0006.jpg
Extracted extracted_frames\frame_0007.jpg
Extracted extracted_frames\frame_0008.jpg
Extracted extracted_frames\frame_0009.jpg
Extracted extracted_frames\frame_0010.jpg
Extracted extracted_frames\frame_0011.jpg
Extracted extracted_frames\frame_0012.jpg
Extracted extracted_frames\frame_0013.jpg
Extracted extracted_frames\frame_0014.jpg
Extracted extracted_frames\frame_0015.jpg
Extracted extracted_frames\frame_0016.jpg
Extracted extracted_frames\frame_0017.jpg
Extracted extracted_frames\frame_0018.jpg
Extracted extracted_frames\frame_0019.jpg
Extracted extracted_frames\frame_0020.jpg
Extracted extracted_frames\frame_0021.jpg
Extracted extracted_frames\frame_0022.jpg
Extracted extracted_frames\frame_0

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping

# a) Load the dataset
# Assuming the Horse or Human dataset is organized in subfolders (train and validation)
from google.colab import drive
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/horse-or-human/horse-or-human'

# Define the paths to the horse and human image folders
horse_dir = os.path.join(base_dir, 'horses')
human_dir = os.path.join(base_dir, 'humans')

# Load images from both classes (horse and human)
horse_images = [os.path.join(horse_dir, f) for f in os.listdir(horse_dir)]
human_images = [os.path.join(human_dir, f) for f in os.listdir(human_dir)]

# Show the number of images in each class
print(f"Number of horse images: {len(horse_images)}")
print(f"Number of human images: {len(human_images)}")

# Plot some images from both classes
fig, axes = plt.subplots(1, 4, figsize=(12, 6))

# Display a few horse images
for i, img_path in enumerate(horse_images[:2]):
    img = load_img(img_path, target_size=(128, 128))
    img_array = img_to_array(img)
    axes[i].imshow(img_array.astype('uint8'))
    axes[i].set_title(f"Horse Image {i+1}")
    axes[i].axis('off')

# Display a few human images
for i, img_path in enumerate(human_images[:2]):
    img = load_img(img_path, target_size=(128, 128))
    img_array = img_to_array(img)
    axes[i+2].imshow(img_array.astype('uint8'))
    axes[i+2].set_title(f"Human Image {i+1}")
    axes[i+2].axis('off')

plt.show()

# Image Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Set up data generators for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# Define the CNN model (using ResNet for transfer learning)
model = Sequential([
    ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=train_generator,
    validation_steps=50,
    callbacks=[early_stopping]
)

# Show the training and testing accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.legend()

plt.show()
